# Nubank case

## Assumptions


In email backlogs, the arrival date refers to the begining of the week (since every date there refers to a Monday).

## Headers

### Imports

In [ ]:
import pandas as pd

### Parameters

### Functions

In [ ]:
def enhance_datetime(df, dt_col_name):
    '''Given a dataframe and a date/datetime column, decompose the column in its components'''
    df[dt_col_name+'__YEAR'] = df[dt_col_name].dt.year
    df[dt_col_name+'__MONTH'] = df[dt_col_name].dt.month
    df[dt_col_name+'__DAY'] = df[dt_col_name].dt.day
    df[dt_col_name+'__WDAY'] = df[dt_col_name].dt.weekday_name.apply(lambda x: x[:3])
    
    return df

### Common data load

In [ ]:
efile = pd.read_excel('./dataset.xlsx', sheet_name=None)
efile.keys()

In [ ]:
hourly = pd.read_excel('./dataset.xlsx', sheet_name='hourly', header=2)
hourly
h = pd.melt(hourly, id_vars='hour/shift')
h['channel'] = h['variable'].apply(lambda x: x.split('-')[0])
h['day'] = h['variable'].apply(lambda x: x.split('-')[1])
#h['time'] = pd.to_datetime(h['hour/shift'], format= '%H:%M:%S').dt.time #Unnecessary, it is already recogizing
print(h.dtypes)
#h.loc[0,'time']
hourly=h
hourly[:5]

## Data preparation

### Compose hourly dataset

Our goal here is to relate all relevant information through their temporal dimension.
Some data will need to be upsampled, since we have monthly, weekly and hourly data.

Let's put everything in a common hourly frequency. Upsampling assumptions must be explicit.

At the end of this section, we should have one unique dataframe with hourly frequency containing number of jobs (upsampled considering daily and hourly distribution percentage) and emaill backlogs (upsampled considering daily and hourly distribution percentage).

The final frame will still keep data about avg job time and p75 of FRT, but this data does not need resampling.

#### Data normalization

Normalize all dates and times, weekday names, squads and channel names, etc.

In [ ]:
job_count = efile['Job count']
# Month end of date
job_count['job_count_date_ME'] = pd.to_datetime(job_count['month'], format='%Y-%m') + pd.offsets.MonthEnd(0)
job_count['channel'] = job_count['channel'].replace('inbound_call', 'phone')
job_count[:5]

In [ ]:
daily = efile['daily']
daily.day.unique()
daily[:5]

In [ ]:
email_blog = efile['Email Backlog']
# Since the arrival date is a Monday, this date refers to the begining of the arrival week
email_blog['begin_arrived_week_date'] = pd.to_datetime(email_blog['arrived week']).dt.tz_localize(None)
email_blog = enhance_datetime(email_blog, 'begin_arrived_week_date')
email_blog[:5]

##### Sanity checks and parameters gathering
Test quality and gather important internal parameters (like minimum and maximum dates)

In [ ]:
assert set(hourly['day'])==set(daily['day'])
assert set(hourly['channel'])==set(job_count['channel'])==set(daily['channel'])
assert set(job_count['squad'])==set(email_blog['squad'])

In [ ]:
print(set(hourly['day']))

In [ ]:
print(set(hourly['channel']))

In [ ]:
MIN_DATE = job_count['job_count_date_ME'].min() + pd.offsets.MonthBegin(-1)
MAX_DATE = job_count['job_count_date_ME'].max()

In [ ]:
MIN_DATE_email = email_blog['begin_arrived_week_date'].min() + pd.offsets.MonthBegin(-1)
MAX_DATE_email = email_blog['begin_arrived_week_date'].max()

In [ ]:
assert MIN_DATE<=MIN_DATE_email
assert MAX_DATE<=MAX_DATE_email

In [ ]:
total_jobs_by_month = job_count[['channel', 'squad', 'job_count_date_ME', 'jobs']].groupby(by=['squad','channel', 'job_count_date_ME'])
if total_jobs_by_month.sum().shape[0]==job_count.shape[0]:
    print('The job_count totals by squad and channel do correspond to grouping them.')
    print('That means, there is no repeated date in the job_count table.')


#### Create datetime df

This will serve as the base frame to relate every info to a time dimension.

In [ ]:
# Generate date dataframe
dt_df = pd.DataFrame()
dt_df['date'] = pd.date_range(MIN_DATE, MAX_DATE)
dt_df['date_ME'] = dt_df['date'] + pd.offsets.MonthEnd(0)

assert (dt_df['date_ME'].dt.month == dt_df['date'].dt.month).all()

dt_df = enhance_datetime(dt_df, 'date')
dt_df[:5]

#### Create month to daily dataframe

For estimating daily jobs (upsample).

DETAILS BELOW

By upsampling, since not all months have exaclty four weeks, it is necessary to normalize the daily percentages to monthly percentages, which may generate some difference in weekday distribution for weeks that are split in two months. But given the growth rate of jobs, this difference is neglectible and the normalization is a reasonable assumption. If if the number of jobs doubles from one month to another, we would be speaking of this increase diluted in 3 or 4 days in 30.

The alternative would be solving a linear system for daily jobs, given monthly totals and in-week percentage distribution as equations.

In [ ]:
# Merge with daily
df_daily = dt_df.merge(daily, how='left',left_on='date__WDAY', right_on='day')
df_daily = df_daily.rename(columns={'percentage':'wday_percentage'})
df_daily[:5]

# Group by channel, sum percentages for the month and normalize channel percentage by total channel monthly percentage
temp = df_daily[['channel','date_ME','wday_percentage']].groupby(by=['date_ME', 'channel']).sum()
temp = temp.reset_index().rename(columns={'wday_percentage':'total_month_wday_percentage_by_channel'})
temp[:5]

# Join totals df_daily and adjust percentage
df_daily = df_daily.merge(temp, how='left', left_on=['channel','date_ME'], right_on=['channel','date_ME'])
df_daily['month_to_day_percentage_adjusted'] = df_daily['wday_percentage']/df_daily['total_month_wday_percentage_by_channel']
df_daily[:5]

# Assert total percentages is summing up to 1 for each channel every month
res = set((df_daily[['date_ME','channel','month_to_day_percentage_adjusted']]
 .groupby(by=['date_ME', 'channel']).sum().round()['month_to_day_percentage_adjusted']))
assert len(res)==1
assert list(res)[0]==1

#### Estimate daily jobs (upsample)

Join the job_counts to the date dataframe (df) and resample the monthly total jobs to daily jobs.

In [ ]:
df_daily_jobs = df_daily.merge(job_count, how='left', left_on=['channel'], right_on=['channel'])
df_daily_jobs = df_daily_jobs.rename(columns={'jobs':'jobs_monthly'}) # for the sake of clarity
df_daily_jobs['jobs_daily'] = df_daily_jobs['month_to_day_percentage_adjusted']*df_daily_jobs['jobs_monthly']
df_daily_jobs[:5]

#### Estimate daily email back logs (upsample)

Join the email_blog to the df_daily_jobs and resample the weekly total jobs to daily backlog, by squad.

In [ ]:
# Join first with datetime
email_blog['channel'] = 'email'
templ = []
for squad in set(email_blog['squad']):
    sli = email_blog[email_blog['squad']==squad].copy()
    temp = dt_df.merge(sli, how='left', left_on=['date'], right_on=['begin_arrived_week_date'])
    temp = temp.ffill()
    temp = temp.rename(columns={'backlog volume':'backlog_volume_of_week'})
    templ.append(temp.copy())

templ = pd.concat(templ)
templ = templ.dropna(subset=['backlog_volume_of_week'])
templ = templ[['date', 'arrived week', 'squad', 'backlog_volume_of_week', 'begin_arrived_week_date','channel']]
templ[-5:]

In [ ]:
# Now join to daily jobs
df = df_daily_jobs.merge(templ, how='left', on=['date', 'squad', 'channel'])
df['backlog_volume_daily'] = df['backlog_volume_of_week']*df['wday_percentage']
df.tail()